In [1]:
from table_extractor import table_extractor
import pandas as pd

In [2]:
import duckdb

In [6]:
from table_extractor import table_extractor


id = "ES1156021-ES903925-ES1305165"
PDF_FILE = f"/OTTO-Project/EMMA_OFFICIAL_STATEMENT_TEMP_PDF/{id}.pdf"

extractor = table_extractor.TableExtractor(PDF_FILE)

df_map = extractor.extract_tables(page_range=[1])

with open(f'/home/factentry/otto_ml/src/antlr/mondal/pdf_table_extractor/{id}.csv', 'w+') as f:
    print(f'"File Name","{PDF_FILE}"', file=f)

17.037037037037038
Skipping small bbox
3.3884297520661155
17.037037037037038
Skipping small bbox
2.7851239669421486
3.8095238095238093
Area: (254, 101, 375, 511) of page no 2


Oct 09, 2024 5:37:27 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Oct 09, 2024 5:37:27 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>


[                 Principal Unnamed: 1 Unnamed: 2   CUSIP
0    (1)Year Amount Coupon      Yield      Price  021807
1  12/1/2019 $5,000 3.000%     2.050%   101.380%     KL5
2  12/1/2020 $5,000 3.000%     2.200%   101.922%     KM3
3  12/1/2021 $5,000 3.000%     2.300%   102.330%     KN1
4  12/1/2022 $5,000 3.000%     2.450%   102.321%     KP6
5  12/1/2023 $5,000 3.000%     2.600%   102.031%     KQ4
6  12/1/2024 $5,000 3.000%     2.700%   101.773%     KR2]


In [4]:
cursor = duckdb.connect("/home/factentry/otto_ml/src/antlr/mondal/pdf_table_extractor/extraction.db")
cursor.sql("DROP TABLE extraction")
cursor.sql("CREATE TABLE IF NOT EXISTS extraction (file_id VARCHAR)")

In [5]:
def table_inserter(cursor, df, file_id):
    for header in df.columns.values:
        cursor.execute(f"""AlTER TABLE extraction ADD COLUMN IF NOT EXISTS "{header.strip()}" VARCHAR""")
    
    cursor.execute(f"INSERT INTO extraction BY NAME SELECT '{file_id.strip()}' as file_id, * FROM df")

In [6]:
def validate_header_weight(headers):
    ALLOWED_HEADERS = [
        "maturity",
        "redemption",
        "date",
        "year",
        "due",
        "principal",
        "amount",
        "price",
        "interest",
        "rate",
        "yield",
        "cusip"
    ]

    unmatched = []
    match_count = 0

    for header in headers:
        if any(kw in header.lower().strip() for kw in ALLOWED_HEADERS):
            match_count += 1
        else:
            unmatched.append(header)
    
    return len(headers) == match_count, unmatched



def standardize_table(df: pd.core.frame.DataFrame, file_id):
    all_headers_allowed, unmatched_headers = validate_header_weight(df.columns.values)
    if all_headers_allowed:
        table_inserter(cursor, df, file_id)

In [7]:
# cursor.sql("COPY (SELECT * FROM extraction) TO '/home/factentry/otto_ml/src/antlr/mondal/pdf_table_extractor/extraction.parquet' (FORMAT 'parquet');")

In [8]:
def infer_text_pattern(str):
    str_pattern=[]
    literal_identifier_map = {
        "-": "DASH",
        "$": "DOLLAR",
        "%": "PERCENT",
        ",": "COMMA",
        ".": "DOT",
    }
    for c in str:
        if c.isdigit():
            str_pattern.append("DIGIT")
        elif c.isspace():
            str_pattern.append("SPACE")
        elif c.isalpha():
            str_pattern.append("CHAR")
        elif c in literal_identifier_map:
            str_pattern.append(literal_identifier_map[c])
        else:
            str_pattern.append("SPLCHAR")
    return str_pattern


print(infer_text_pattern("2024"))

['DIGIT', 'DIGIT', 'DIGIT', 'DIGIT']


In [9]:
import re

In [10]:
from dateutil import parser

dt = parser.parse("7-1-2034")
print(dt)

2034-07-01 00:00:00


In [11]:
import string


def can_be_currency(str):
    if str[0] == "$":
        return True
    elif re.sub(r'[,\.\*]', '', str).isnumeric() and len(str) > 3:
        return True
    return False

def try_parse_date(str):
    if len(str) < 4:
        return None
    try:
        return parser.parse(str.replace(' ', ''))
    except:
        return None

def can_be_rate(str):
    return re.sub(r'[%\*\.]', '', str).isnumeric()

def can_be_cusip_part(str):
    str = re.sub(r'[\@\* ]', '', str)
    return len(str) == 3 and str[-1].isdigit()

def can_be_cusip(str):
    str = re.sub(r'[\@\* ]', '', str)
    return len(str) == 9 and can_be_cusip_part(str[-3:])

def infer_type(str):
    str = str.encode('ascii', errors='ignore').decode() # Filter out non-ascii characters for now
    if not str or str.lower() == "nan":
        return "NAN"
    elif str.replace('.', '', 1).replace('-', '', 1).isnumeric():
        return "NUMBER"
    elif str.isalpha():
        return "STRING"
    elif str.isascii():
        if len(str.translate(str.maketrans('', '', string.punctuation + string.whitespace))) > 15:
            return "SENTENCE"
        elif can_be_currency(str):
            return "CURRENCY"
        elif try_parse_date(str):
            return "DATE"
        elif can_be_rate(str):
            return "RATE"
        elif can_be_cusip(str):
            return "CUSIP"
        elif can_be_cusip_part(str):
            return "CUSIP_PART"
        
    return " ".join(infer_text_pattern(str))


print(infer_type("2024"))

NUMBER


In [12]:
COLUMN_DTYPE_MAP = {
    "and": ["ANY"],
    "cusip": ["CUSIP", "CUSIP_PART"],
    "date": ["DATE", "SENTENCE"],
    "maturity": ["DATE", "SENTENCE"],
    "rate": ["RATE", "NUMBER"],
    "yield": ["RATE", "NUMBER"],
    "amount": ["CURRENCY"]
}

In [13]:
import itertools


def clean_df(df):
    is_multiline_header = False
    # Scan for data type column wise
    for column in df.columns:
        column_value_dtypes = [(k, len(list(g))) for k, g in itertools.groupby([infer_type(str(x)) for x in df[column]])]
        if len(column_value_dtypes) == 2:
            if column_value_dtypes[0][1] <= 2:
                is_multiline_header = True
            else:
                is_multiline_header = False
    
    all_null_columns = df.isnull().values.all(axis=0)

    all_nan_ranges = []
    consecutive_nan_range = (None, None)
    for idx, col in enumerate(df.columns):
        if col.lower() == "nan" or "unnamed" in col.lower():
            next_col_name = df.columns[min(idx + 1, len(df.columns) - 1)].lower()
            if next_col_name != "nan" or "unnamed" not in next_col_name:
                if not consecutive_nan_range[0]:
                    consecutive_nan_range = (idx, None)
            else:
                consecutive_nan_range = (consecutive_nan_range[0], idx)
                all_nan_ranges.append(consecutive_nan_range)
                consecutive_nan_range = (None, None)
    

        
    if is_multiline_header:
        current_headers = ["" if header.lower().startswith("unnamed") else header for header in df.columns.tolist()]
        first_row_values = df.iloc[0].tolist()
        new_headers = [f"{header} {value}".strip() for header, value in zip(current_headers, first_row_values)]
        df.columns = new_headers
        df = df.drop(0)
    
    return df

In [14]:
# Test main impl

import muni_table_standardiser as standardiser

In [15]:
for page_no, dfs in df_map.items():
    for df in dfs:
        if df.shape[1] == 1:
                continue
        if not df.shape[0]:
            continue

        if len(df.iloc[0]) < 2:
            continue

        row_count = df.shape[0]

        is_line = False

        for ri in range(row_count):
            words = str(df.iloc[ri][0]).split()
            avg_word_length = sum(len(word) for word in words) / len(words)
            if len(words) > 10 and avg_word_length > 5:
                is_line = True
                continue
        
        if is_line:
            continue
        with open(f'/home/factentry/otto_ml/src/antlr/mondal/pdf_table_extractor/{id}.csv', 'a') as f:
            print(f'\n', file=f)

            df = standardiser.clean_df(df)
            header_list = list(df.columns.values)
            header_list = ["" if header.lower().startswith("unnamed") or header.lower().startswith("nan") else header for header in header_list]
            # standardize_table(df, id)

            # Standardize headers
            # current_headers = ["" if header.lower().startswith("unnamed") else header for header in df.columns.tolist()]
            # first_row_values = df.iloc[0].tolist()
            # new_headers = [f"{header} {value}".strip() for header, value in zip(current_headers, first_row_values)]
            # df.columns = new_headers
            # df = df.drop(0)

            print(df.head())
            df.to_csv(f, index=False, header=header_list)

WARN: Found column type of ANY. Considering as freezed column
!!!! Merge success !!!!
WARN: Found column type of ANY. Considering as freezed column
!!!! Merge success !!!!
WARN: Found column type of ANY. Considering as freezed column
!!!! Merge success !!!!
WARN: Found column type of ANY. Considering as freezed column
!!!! Merge success !!!!
WARN: Found column type of ANY. Considering as freezed column
!!!! Merge success !!!!
WARN: Found column type of ANY. Considering as freezed column
!!!! Merge success !!!!
WARN: Found column type of ANY. Considering as freezed column
!!!! Merge success !!!!
WARN: Found column type of ANY. Considering as freezed column
!!!! Merge success !!!!
WARN: Found column type of ANY. Considering as freezed column
!!!! Merge success !!!!
WARN: Found column type of ANY. Considering as freezed column
WARN: Found column type of ANY. Considering as freezed column
WARN: Found column type of ANY. Considering as freezed column
WARN: Found column type of ANY. Consider

In [16]:
import string


len("U.S. Bank Trust National Association, New York, New York.".translate(str.maketrans('', '', string.punctuation + string.whitespace)))

44

In [17]:
print(infer_type("U.S. Bank Trust National Association, New York, New York."))

SENTENCE


In [18]:
test_str_list = ["3",   "4",    "1 5 3 Q E D",   "4"  , "NaN"  ,"June 1, 2018",       "$6,845,000"]

In [19]:
test_str_list

['3', '4', '1 5 3 Q E D', '4', 'NaN', 'June 1, 2018', '$6,845,000']

In [20]:
list(map(lambda x: x if x.lower().strip() not in ["nan", "none", "null"] else "", test_str_list))

['3', '4', '1 5 3 Q E D', '4', '', 'June 1, 2018', '$6,845,000']

In [21]:
test_str_list = list(map(lambda x: x if x.lower().strip() not in ["nan", "none", "null"] else "", test_str_list))

In [22]:
infr_type_list = [infer_type(e) for e in test_str_list]

In [23]:
test_expected_type = [None, None, "CUSIP", None, None, "DATE", "CURRENCY"]

In [24]:
freezed_columns = []
missmatch_cols = {}

for col_idx, (exp_type, inferred_type) in enumerate(zip(test_expected_type, infr_type_list)):
    if not exp_type:
        missmatch_cols[col_idx] = "U"
    elif exp_type == inferred_type:
        freezed_columns.append(col_idx)
    elif exp_type != inferred_type:
        missmatch_cols[col_idx] = "M"

In [25]:
freezed_columns

[5, 6]

In [26]:
missmatch_cols

{0: 'U', 1: 'U', 2: 'M', 3: 'U', 4: 'U'}

In [27]:
test_str_list[1:]

['4', '1 5 3 Q E D', '4', '', 'June 1, 2018', '$6,845,000']

In [28]:
start_idx = -1
end_idx = -1
last_mismatch_idx = -1
merged_column_contents = {}
merge_region = {}
for col_idx, miss_type in missmatch_cols.items():
    if miss_type == "U":
        if start_idx == -1:
            start_idx = col_idx
        if end_idx == -1:
            if missmatch_cols[min(col_idx + 1, len(missmatch_cols) - 1)] == "M":
                end_idx = col_idx
    elif miss_type == "M":
        merged_column_contents[col_idx] = " ".join(test_str_list[start_idx:end_idx+1] + [test_str_list[col_idx]])
        merge_region[col_idx] = (start_idx, end_idx)
        start_idx = -1
        end_idx = -1
        last_mismatch_idx = col_idx

print(merge_region)

# No match found on left side. Merge <- direction
if end_idx == -1:
    merged_column_contents[last_mismatch_idx] = (merged_column_contents[last_mismatch_idx] + " " + " ".join(test_str_list[start_idx:list(missmatch_cols.keys())[-1] + 1])).strip()
    merge_region[last_mismatch_idx] = (merge_region[last_mismatch_idx][0], list(missmatch_cols.keys())[-1])

merged_column_contents, 

{2: (0, 1)}


({2: '3 4 1 5 3 Q E D 4'},)

In [29]:
for col_idx, content in merged_column_contents.items():
    if infer_type(content) == test_expected_type[col_idx]:
        print("!!!! Merge success !!!!")
        # do actual merge on df

!!!! Merge success !!!!


In [1]:
import psutil

In [2]:
psutil.cpu_count()

104

In [5]:
(psutil.cpu_count() // 100) * 80

80

In [7]:
psutil.virtual_memory()

svmem(total=541014310912, available=306733891584, percent=43.3, used=231291240448, free=308231770112, active=112231190528, inactive=116548583424, buffers=78376960, cached=1412923392, shared=413696, slab=1241886720)

In [6]:
(psutil.virtual_memory().free // 100) * 70

215721493260

In [2]:
from pymongo import MongoClient

In [3]:

mongo_client = MongoClient('mongodb://192.168.1.245:27017/?ssl=false')
mongo_db = mongo_client['otto_ml']
mongo_collection = mongo_db['muni_tables']

In [4]:
mongo_collection.insert_many(df.to_dict('records'))

Collection(Database(MongoClient(host=['192.168.1.245:27017'], document_class=dict, tz_aware=False, connect=True, tls=False), 'otto_ml'), 'muni_tables')